In [26]:
from datetime import datetime
import logging
import os
import time
import uuid

import dotenv
import labelbox as lb
import tenacity

dotenv.load_dotenv(dotenv_path="./.env")


True

## Setup

In [27]:
logging.basicConfig(level=logging.INFO)

In [28]:
LABELBOX_API_KEY = os.getenv('LABELBOX_API_KEY')

Establish our date time object for testing

In [29]:
test_video_path = "./big_buck_bunny_snippet.mp4"

example_date = datetime.fromisoformat('2023-08-07 09:00:05.450+00:00')
logging.info(f"Example date (notice the milliseconds): '{example_date}'")

INFO:root:Example date (notice the milliseconds): '2023-08-07 09:00:05.450000+00:00'


## Config Labelbox client

In [30]:
client = lb.Client(api_key=LABELBOX_API_KEY)

INFO:labelbox.client:Initializing Labelbox client at 'https://api.labelbox.com/graphql'


## Create schema w/ CustomMetadataDateTime field

In [31]:
mdo = client.get_data_row_metadata_ontology()


def is_valid_metadata(name: str):
    try:
        return mdo.get_by_name(name=name)
    except:
        return None

meta_data_field = is_valid_metadata(name='datetime_test')
if meta_data_field is not None:
    logging.info("datetime_test metadata field already exists")
    logging.info(f"datetime_test field type = {meta_data_field.kind}")
    if meta_data_field.kind != lb.schema.data_row_metadata.DataRowMetadataKind.datetime:
        raise ValueError("Invalid test, the datetime_test should be DataRowMetadataKind")
else:
    metadata_schema_video_start = mdo.create_schema(
        name='datetime_test',
        kind=lb.schema.data_row_metadata.DataRowMetadataKind('CustomMetadataDateTime')
    )
    logging.info("Created the datetime_test metadata field")


INFO:root:datetime_test metadata field already exists
INFO:root:datetime_test field type = DataRowMetadataKind.datetime


## Prepare single video datarow with time field/value

In [32]:
datarows = list()

In [33]:
@tenacity.retry(stop=tenacity.stop_after_attempt(3))
def upload_video_wrapper(video_path):
    return client.upload_file(video_path)


logging.info(f"Uploading video file...")
video_url = upload_video_wrapper(test_video_path)
logging.info("Uploaded video file")

uid = str(uuid.uuid4())
datarows.append({
    lb.DataRow.row_data: video_url,
    lb.DataRow.external_id: uid,
    lb.DataRow.global_key: uid,
    lb.DataRow.metadata_fields: [
        lb.DataRowMetadataField(name='datetime_test', value=example_date)
    ]
})

logging.info(f"Datarows staged with CustomMetadataDateTime field set to {example_date}")

INFO:root:Uploading video file...
INFO:root:Uploaded video file
INFO:root:Datarows staged with CustomMetadataDateTime field set to 2023-08-07 09:00:05.450000+00:00


## Create the Dataset

In [34]:
dataset_name = "Test Labelbox MetaData Field"

dataset = None
try:
    dataset = client.get_datasets(where=lb.Dataset.name == dataset_name).get_one()
    
    logging.info("Dataset already exists, clearing the dataset's datarow for testing purposes")
    lb.DataRow.bulk_delete(data_rows=list(dataset.data_rows()))
except Exception as e:
    pass

if dataset is None:
    dataset = client.create_dataset(
        iam_integration=None,
        name=dataset_name,
        description=f"A simple test dataset for experimenting with Labelbox's date metadata field"
    )
    logging.info("Created dataset")

dataset

INFO:root:Dataset already exists, clearing the dataset's datarow for testing purposes


<Dataset ID: cll1g23du08ht073m84j4avt1>

## Upload the video (the datarows) to the Dataset

In [35]:
create_task = dataset.create_data_rows(datarows)

In [36]:
create_task.wait_till_done()

In [37]:
create_task.status

'COMPLETE'

In [38]:
create_task.errors

In [39]:
logging.info("Sleeping for 30 seconds so we can wait for things to sync on Labelbox servers")
time.sleep(30)

INFO:root:Sleeping for 30 seconds so we can wait for things to sync on Labelbox servers


## Export the dataset and prepare to check the date value

In [40]:
# set the export params to include/exclude certain fields
export_params={
    "attachments": True,
    "metadata_fields": True,
    "data_row_details": True,
    "project_details": True,
    "performance_details": True,
}

# get a dataset
dataset = client.get_dataset(dataset.uid)

# run the export task
export_task = dataset.export_v2(params=export_params)
export_task.wait_till_done()

# view errors and results
if export_task.errors:
    logging.error(export_task.errors)

export_json = export_task.result
logging.info(export_json)

INFO:root:[{'data_row': {'id': 'cll2hxk4g0071074u0q81g783', 'external_id': 'f90e94d3-f21f-4722-8e2c-50ccce1fc8c5', 'global_key': 'f90e94d3-f21f-4722-8e2c-50ccce1fc8c5', 'row_data': 'https://storage.labelbox.com/ck94eylfhyxkp0976giowy90g%2Ff9f42846-9726-93f2-4e6e-2b6dcc502b94-big_buck_bunny_snippet.mp4?Expires=1691597360286&KeyName=labelbox-assets-key-3&Signature=Zh4o-e0C-iBMe3Q7xEMITGsAHo4', 'details': {'dataset_id': 'cll1g23du08ht073m84j4avt1', 'dataset_name': 'Test Labelbox MetaData Field', 'created_at': '2023-08-08T16:08:46.000+00:00', 'updated_at': '2023-08-08T16:08:46.000+00:00', 'last_activity_at': '2023-08-08T16:08:46.698+00:00', 'created_by': 'ben.talberg@wildflowerschools.org'}}, 'media_attributes': {'height': 720, 'width': 1280, 'mime_type': 'video/mp4'}, 'attachments': [], 'metadata_fields': [{'schema_id': 'cll1eyeqg07ah073md56ifiy8', 'schema_name': 'datetime_test', 'value': '2023-08-07T09:00:05.000+00:00'}]}]


## Test

In [41]:
datetime_test_field = export_json[0]['metadata_fields'][0]

datetime_test_field_value_raw = datetime_test_field['value']
datetime_test_field_value_as_date = datetime.fromisoformat(datetime_test_field_value_raw)

Assert the `example_date` value we started with equals the value we fetched from Labelbox

In [42]:
logging.info(f"Expect the meta data date field to contain millisecond information")
assert datetime_test_field_value_as_date == example_date


INFO:root:Expect the meta data date field to contain millisecond information


AssertionError: 

Assert if we round off the `example_date`'s milliseconds it DOES NOT equal the value we fetched from Labelbox

In [43]:
rounded_example_date = example_date.replace(microsecond=0)

logging.info(f"Expect that if round off the date ({rounded_example_date}) it will NOT match what we receive from Labelbox")
assert datetime_test_field_value_as_date != rounded_example_date

INFO:root:Expect that if round off the date (2023-08-07 09:00:05+00:00) it will NOT match what we receive from Labelbox


AssertionError: 

In [44]:
logging.info(f"Date value with microseconds that we sent to Labelbox: {example_date}")
logging.info(f"Date value we found on Labelbox (raw): {datetime_test_field_value_raw}")
logging.info(f"Date value we found on Labelbox (parsed): {datetime_test_field_value_as_date}")

INFO:root:Date value with microseconds that we sent to Labelbox: 2023-08-07 09:00:05.450000+00:00
INFO:root:Date value we found on Labelbox (raw): 2023-08-07T09:00:05.000+00:00
INFO:root:Date value we found on Labelbox (parsed): 2023-08-07 09:00:05+00:00
